In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Carregar os dados
file_path = 'https://raw.githubusercontent.com/CD-CrashTest/Crash_Test/refs/heads/main/scrappingNewDf.csv'  # Substitua pelo caminho do seu arquivo
data = pd.read_csv(file_path)

# Remover 'kg' da coluna 'Kerb Weight' e converter para numérico
data['Kerb Weight'] = data['Kerb Weight'].str.replace('kg', '').astype(float)

data.head()

,Tested Model,Body Type,Year Of Publication,Kerb Weight,VIN From Which Rating Applies,Class,Frontal Offset Deformable Barrier,Whiplash Rear Impact,AEB City,Lateral Impact,...,Occupant Status Monitoring,AEB Car-to-Car,Rescue and Extrication.1,Crash Test Performance based on 6 & 10 year old children.1,Safety Features.1,CRS Installation Check.1,Pedestrian & Cyclist Head,Pelvis,Femur,Knee & Tibia
0,AUDI A3 Sportback e-tron 1.4 TFSI + e-Engine '...,- 5 door hatchback,2014,1607.0,- applies to all A3 Sportback e-trons of the s...,Small Family Car,14.7,2.3,0.0,14.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"BMW 2 Series Active Tourer 1.5 Base, LHD",- 5 door hatchback,2014,1320.0,- applies to all 2 Series of the specification...,Small Family Car,11.3,2.5,2.1,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Citroën Berlingo 1.6 diesel 'Confort', LHD",- 5 door hatchback,2014,1431.0,- VF7 7J9HP0 EJ713660,Small MPV,11.6,1.8,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Toyota Aygo 1 x-play, LHD",- 5 door hatchback,2014,874.0,- applies to all C1s of the specification tested,City and Supermini,12.5,2.6,0.0,15.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Citroën C4 CACTUS 1.2 'FEEL', LHD",- 5 door hatchback,2014,965.0,- applies to all C4 Cactus of the specificatio...,Small Family Car,14.4,2.7,0.0,14.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Preencher valores ausentes nas colunas numéricas com valores aleatórios entre média ± desvio padrão
numeric_cols = data.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    mean = data[col].mean()
    std_dev = data[col].std()
    data[col] = data[col].apply(lambda x: np.random.uniform(mean - std_dev, mean + std_dev) if pd.isna(x) else x)

In [ ]:
# Separar as features e a variável alvo
features = data.select_dtypes(include=[np.number]).drop(columns=['rating'], errors='ignore')
target = data['rating']

In [ ]:
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

,Year Of Publication,Kerb Weight,Frontal Offset Deformable Barrier,Whiplash Rear Impact,Lateral Impact,18 months old child,36 months old child,VRU Impact Protection,Crash Test Performance,Safety Features,...,Occupant Status Monitoring,AEB Car-to-Car,Rescue and Extrication.1,Crash Test Performance based on 6 & 10 year old children.1,Safety Features.1,CRS Installation Check.1,Pedestrian & Cyclist Head,Pelvis,Femur,Knee & Tibia
6,2014,1216.0,8.300000,1.100000,12.6,12.000000,9.100000,20.000000,21.100000,4.0,...,1.958284,5.816923,1.245710,21.895324,6.685318,12.136804,12.026327,3.940784,4.119975,8.913956
104,2012,1088.0,14.200000,2.700000,14.9,11.000000,10.500000,22.000000,21.500000,5.0,...,2.902437,4.631148,3.335981,23.554735,6.667881,12.019594,11.643076,4.156087,3.590475,9.253301
114,2012,845.0,10.800000,1.100000,12.8,8.600000,12.000000,19.200000,20.600000,3.0,...,2.609441,5.114236,3.278168,24.763705,7.511725,11.983300,12.964278,4.048741,3.613902,8.246387
493,2021,2103.0,8.870012,2.334696,15.5,11.609253,11.446525,24.624268,22.138181,7.0,...,1.000000,5.600000,2.267448,22.502092,7.323097,11.700259,12.839462,2.857747,4.911203,8.398716
414,2019,1240.0,6.800000,1.400000,14.0,10.802382,9.742727,24.953154,23.774056,7.0,...,3.129881,4.142721,3.927087,22.470305,5.823453,12.097472,11.708685,2.944136,4.491581,9.240806


In [ ]:
y_train.head()

,rating
6,3
104,5
114,3
493,5
414,5


In [ ]:
# Inicializar e treinar o classificador Random Forest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Fazer previsões e avaliar o modelo
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Exibir resultados
print("Acurácia:", accuracy)
print("Relatório de Classificação:\n", report)

Acurácia: 0.8582677165354331
Relatório de Classificação:
               precision    recall  f1-score   support

           2       0.00      0.00      0.00         1
           3       0.57      0.44      0.50         9
           4       0.75      0.52      0.62        23
           5       0.89      0.99      0.94        94

    accuracy                           0.86       127
   macro avg       0.55      0.49      0.51       127
weighted avg       0.84      0.86      0.84       127



In [ ]:
# Kerb Weight
# Safety Features
# CRS Installation Check
# Class
# Seatbelt Reminder

# Selecionar as features relevantes
features_selected = ['Kerb Weight', 'Safety Features', 'CRS Installation Check', 'Class', 'Seatbelt Reminder']
features = data[features_selected]

# Converter features categóricas para numéricas usando one-hot encoding
categorical_features = ['Class', 'Safety Features', 'CRS Installation Check', 'Seatbelt Reminder'] #Especificar as colunas categoricas
numerical_features = ['Kerb Weight'] #Especificar as colunas numericas

# Converter as features categóricas para numéricas usando one-hot encoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = encoder.fit_transform(features[categorical_features])

# Criar um novo dataframe com as features codificadas
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Concatenar as features numéricas com as codificadas
X = pd.concat([features[numerical_features], encoded_df], axis=1)

# Converter o rating para numérico (se necessário)
y = data['rating'].astype(int)

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados numéricos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criar o modelo
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax')) # 6 classes (0 a 5)

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Acurácia do modelo: {accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.3024 - loss: 1.9091 - val_accuracy: 0.5050 - val_loss: 1.7135
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6833 - loss: 1.2910 - val_accuracy: 0.5941 - val_loss: 1.5086
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7563 - loss: 0.9682 - val_accuracy: 0.5842 - val_loss: 1.3795
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7669 - loss: 0.7714 - val_accuracy: 0.5545 - val_loss: 1.2967
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7649 - loss: 0.6946 - val_accuracy: 0.5446 - val_loss: 1.2300
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7944 - loss: 0.5819 - val_accuracy: 0.5347 - val_loss: 1.1828
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8020 - loss: 0.5218 - val_accuracy: 0.5446 - val_loss: 1.1613
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7991 - loss: 0.5119 - val_accuracy: 0.5743 - val_loss: 1.1281
E

In [ ]:
# Solicitar ao usuário os valores das features
kerb_weight = float(input("Digite o peso do veículo em Kg: "))
safety_features = input("Digite nota de segurança (0-10): ")
crs_installation = input("Digite a nota de facilidade de instalação de cadeirinha infantil (0-12): ")
vehicle_class = input("Digite a classe do veículo: ")
seatbelt_reminder = input("Veículo possúi alerta de cinto de segurança (sim ou não): ")

# Criar um DataFrame com os valores de entrada do usuário
user_input = pd.DataFrame({
    'Kerb Weight': [kerb_weight],
    'Safety Features': [safety_features],
    'CRS Installation Check': [crs_installation],
    'Class': [vehicle_class],
    'Seatbelt Reminder': [seatbelt_reminder]
})

# Pré-processar os dados de entrada do usuário da mesma forma que os dados de treinamento
categorical_features = ['Class', 'Safety Features', 'CRS Installation Check', 'Seatbelt Reminder']
numerical_features = ['Kerb Weight']

# Converter as features categóricas para numéricas usando one-hot encoding (usando o mesmo encoder treinado antes)
encoded_features = encoder.transform(user_input[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Concatenar as features numéricas com as codificadas
X_user = pd.concat([user_input[numerical_features], encoded_df], axis=1)

# Normalizar os dados de entrada do usuário usando o mesmo scaler usado no treino
X_user_scaled = scaler.transform(X_user)

# Fazer a previsão com o modelo treinado
prediction = model.predict(X_user_scaled)

# Obter a classe prevista (índice da probabilidade máxima)
predicted_class = np.argmax(prediction)

print(f"Previsão de classificação (Rating): {predicted_class}")

Digite o peso do veículo (Kerb Weight em kg): 1000
Digite as características de segurança (Safety Features): 0
Digite a verificação da instalação do CRS (CRS Installation Check): 0
Digite a classe do veículo (Class): Small Family Car
Digite se possui lembrete de cinto de segurança (Seatbelt Reminder - sim ou nao): nao
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Previsão de classificação (Rating): 4
